In [2]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier
from sklearn.metrics import make_scorer, roc_auc_score
df = pd.read_csv("../data/dataset_proyecto_statistical_learning_scaled.csv")
df

,Unnamed: 0,date,targetid,attribute,weekstartdate,gender,agerange,device,creative,placement,...,state_code,utmsource_code,utmcampaign_code,campaigntype_code,phone,calls,voicemail,marketingcost,totalcost,age
0,0,2023-07-01,645cdca06beec953cd8ceef8,Health (cancer),2023-07-01,FEMALE,30 - 39,MOBILE,8984142e-3a16-4599-8f0a-41f897cf2d52,GetItFree - All Traffic - Primary,...,0.024685,0.021840,0.045753,0.454749,0.0,-0.383248,-0.358253,-0.282725,-0.414268,-1.244280
1,1,2023-07-01,645cdca06beec953cd8cef14,Health (cancer: Stage I),2023-07-01,FEMALE,50 - 59,MOBILE,8984142e-3a16-4599-8f0a-41f897cf2d52,Workn Well - Primary Placement,...,0.009572,0.001579,0.001480,0.454749,0.0,-0.383248,-0.358253,-0.282725,-0.414268,-0.737901
2,2,2023-07-01,645cdeab6beec953cd8cef94,Health (cancer: Stage Iv),2023-07-01,FEMALE,60 - 69,MOBILE,33253883-03db-421c-874f-2acb78459fb3,Workn Well - Primary Placement,...,0.011002,0.001579,0.001480,0.454749,0.0,-0.383248,-0.358253,-0.282725,-0.414268,-0.305049
3,3,2023-07-01,645cdeaa6beec953cd8cef84,Health (cancer: Stage Ii),2023-07-01,FEMALE,50 - 59,MOBILE,33253883-03db-421c-874f-2acb78459fb3,Workn Well - Primary Placement,...,0.024965,0.181827,0.280388,0.454749,0.0,-0.383248,-0.358253,-0.282725,-0.414268,-0.430679
4,4,2023-07-01,645cdeab6beec953cd8cef92,Health (cancer: Stage Iii),2023-07-01,FEMALE,60 - 69,MOBILE,33253883-03db-421c-874f-2acb78459fb3,Workn Well - Primary Placement,...,0.024965,0.181827,0.280388,0.454749,0.0,-0.383248,-0.358253,-0.282725,-0.414268,0.242656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60800,60800,2023-02-07,63e2ef3ed1414a450236cfa4,Health (cancer),2023-02-07,FEMALE,30 - 39,MOBILE,b0a3b69c0521fbffc0aca84bec2adacf,PrizeStash - TXM - Primary,...,0.032349,0.198438,0.069484,0.454749,0.0,-0.383248,-0.358253,-0.581051,-0.504320,-1.189325
60801,60801,2023-02-07,63e2ef3ed1414a450236cfa4,Health (cancer),2023-02-07,FEMALE,20 - 29,MOBILE,b0a3b69c0521fbffc0aca84bec2adacf,PrizeStash - TXM - Primary,...,0.015377,0.198438,0.069484,0.454749,0.0,-0.383248,-0.358253,-0.581051,-0.504320,-1.259997
60802,60802,2023-02-07,63e2ef3ed1414a450236cf9a,Health (cancer: Stage Iv),2023-02-07,FEMALE,30 - 39,MOBILE,b0a3b69c0521fbffc0aca84bec2adacf,PrizeStash - TXM - Primary,...,0.024965,0.125450,0.069484,0.454749,0.0,-0.383248,-0.358253,-0.581051,-0.504320,-1.159850
60803,60803,2023-02-07,63e2ef3ed1414a450236cfa2,Health (cancer: Diagnosed),2023-02-07,FEMALE,50 - 59,MOBILE,4ae8788fd57fc376cbfbfd54e98a0c1e,Jetsetty - Primary Placement,...,0.099811,0.027284,0.067018,0.454749,0.0,-0.383248,-0.358253,-0.581051,-0.504320,-0.544164


In [4]:
df = df.select_dtypes(include=np.number)
X = df.drop('unique_leads', axis=1)
y = df['unique_leads']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lista para almacenar los modelos
models = []
scorer = make_scorer(roc_auc_score)

# Gaussian Naive Bayes
pipeline_nb = Pipeline([('scale', StandardScaler()), ('nb', GaussianNB())])
param_grid_nb = {
    'nb__var_smoothing': np.logspace(0,-9, num=100)
}
grid_nb = RandomizedSearchCV(pipeline_nb, param_grid_nb, cv=2, scoring=scorer, n_jobs=-1, n_iter=10)
grid_nb.fit(X_train, y_train)
models.append(('GaussianNB', grid_nb))

# Linear Discriminant Analysis
pipeline_lda = Pipeline([('scale', StandardScaler()), ('lda', LinearDiscriminantAnalysis())])
param_grid_lda = {
    'lda__solver' : ['svd', 'lsqr', 'eigen'],
}
grid_lda = RandomizedSearchCV(pipeline_lda, param_grid_lda, cv=2, scoring=scorer, n_jobs=-1, n_iter=10)
grid_lda.fit(X_train, y_train)
models.append(('LinearDiscriminantAnalysis', grid_lda))

# Logistic Regression
pipeline_lr = Pipeline([('scale', StandardScaler()), ('lr', LogisticRegression(max_iter=10000))])
param_grid_lr = {
    'lr__C': np.logspace(-4, 4, 20),
    'lr__solver': ['liblinear']
}
grid_lr = RandomizedSearchCV(pipeline_lr, param_grid_lr, cv=2, scoring=scorer, n_jobs=-1, n_iter=10)
grid_lr.fit(X_train, y_train)
models.append(('LogisticRegression', grid_lr))

# Support Vector Machine
pipeline_svm = Pipeline([('scale', StandardScaler()), ('svm', SVC())])
param_grid_svm = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': [1, 0.1, 0.01, 0.001],
    'svm__kernel': ['rbf', 'linear', 'poly']
}
grid_svm = RandomizedSearchCV(pipeline_svm, param_grid_svm, cv=2, scoring=scorer, n_jobs=-1, n_iter=10)
grid_svm.fit(X_train, y_train)
models.append(('SVC', grid_svm))

# Decision Tree
pipeline_dt = Pipeline([('scale', StandardScaler()), ('dt', DecisionTreeClassifier())])
param_grid_dt = {
    'dt__criterion' : ['gini', 'entropy'],
    'dt__max_depth' : [4,6,8,10,12],
}
grid_dt = RandomizedSearchCV(pipeline_dt, param_grid_dt, cv=2, scoring=scorer, n_jobs=-1, n_iter=10)
grid_dt.fit(X_train, y_train)
models.append(('DecisionTreeClassifier', grid_dt))

# Random Forest
pipeline_rf = Pipeline([('scale', StandardScaler()), ('rf', RandomForestClassifier())])
param_grid_rf = {
    'rf__n_estimators' : [50, 100, 200],
    'rf__max_features' : ['auto', 'sqrt', 'log2'],
    'rf__max_depth' : [4,5,6,7,8],
    'rf__criterion' :['gini', 'entropy']
}
grid_rf = RandomizedSearchCV(pipeline_rf, param_grid_rf, cv=2, scoring=scorer, n_jobs=-1, n_iter=10)
grid_rf.fit(X_train, y_train)
models.append(('RandomForestClassifier', grid_rf))

# Quadratic Discriminant Analysis
pipeline_qda = Pipeline([('scale', StandardScaler()), ('qda', QuadraticDiscriminantAnalysis())])
param_grid_qda = {
    'qda__reg_param': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0],
}
grid_qda = RandomizedSearchCV(pipeline_qda, param_grid_qda, cv=2, scoring=scorer, n_jobs=-1, n_iter=10)
grid_qda.fit(X_train, y_train)
models.append(('QuadraticDiscriminantAnalysis', grid_qda))

# AdaBoost
pipeline_ab = Pipeline([('scale', StandardScaler()), ('ab', AdaBoostClassifier())])
param_grid_ab = {
    'ab__n_estimators' : [50, 100, 200],
    'ab__learning_rate' : [0.01,0.05,0.1,0.3,1]
}
grid_ab = RandomizedSearchCV(pipeline_ab, param_grid_ab, cv=2, scoring=scorer, n_jobs=-1, n_iter=10)
grid_ab.fit(X_train, y_train)
models.append(('AdaBoostClassifier', grid_ab))

# XGBoost
pipeline_xgb = Pipeline([('scale', StandardScaler()), ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))])
param_grid_xgb = {
    'xgb__n_estimators' : [50, 100, 200],
    'xgb__learning_rate' : [0.01, 0.05, 0.1, 0.3, 1],
    'xgb__max_depth' : [3, 5, 7, 9]
}
grid_xgb = RandomizedSearchCV(pipeline_xgb, param_grid_xgb, cv=2, scoring=scorer, n_jobs=-1, n_iter=10)
grid_xgb.fit(X_train, y_train)
models.append(('XGBClassifier', grid_xgb))

# LGBM
pipeline_lgbm = Pipeline([('scale', StandardScaler()), ('lgbm', LGBMClassifier())])
param_grid_lgbm = {
    'lgbm__n_estimators' : [50, 100, 200],
    'lgbm__learning_rate' : [0.01, 0.05, 0.1, 0.3, 1],
    'lgbm__max_depth' : [3, 5, 7, 9]
}
grid_lgbm = RandomizedSearchCV(pipeline_lgbm, param_grid_lgbm, cv=2, scoring=scorer, n_jobs=-1, n_iter=10)
grid_lgbm.fit(X_train, y_train)
models.append(('LGBMClassifier', grid_lgbm))

# Imprimimos los resultados
for name, model in models:
    print(f"\n{name}")
    print(f"Best Score: {model.best_score_}")
    print(f"Best Parameters: {model.best_params_}")
    y_pred = model.predict(X_test)
    print(f"ROC AUC: {roc_auc_score(y_test, y_pred)}")
    print(f"Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}")

c:\Users\DELL\anaconda3.7\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
c:\Users\DELL\anaconda3.7\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
c:\Users\DELL\anaconda3.7\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")



GaussianNB
Best Score: 0.9998618079768687
Best Parameters: {'nb__var_smoothing': 6.579332246575682e-09}
ROC AUC: 0.9997872521487533
Confusion Matrix: 
[[11746     5]
 [    0   410]]

LinearDiscriminantAnalysis
Best Score: 0.9996890547263682
Best Parameters: {'lda__solver': 'svd'}
ROC AUC: 1.0
Confusion Matrix: 
[[11751     0]
 [    0   410]]

LogisticRegression
Best Score: 1.0
Best Parameters: {'lr__solver': 'liblinear', 'lr__C': 4.281332398719396}
ROC AUC: 1.0
Confusion Matrix: 
[[11751     0]
 [    0   410]]

SVC
Best Score: 1.0
Best Parameters: {'svm__kernel': 'poly', 'svm__gamma': 1, 'svm__C': 10}
ROC AUC: 1.0
Confusion Matrix: 
[[11751     0]
 [    0   410]]

DecisionTreeClassifier
Best Score: 1.0
Best Parameters: {'dt__max_depth': 4, 'dt__criterion': 'gini'}
ROC AUC: 1.0
Confusion Matrix: 
[[11751     0]
 [    0   410]]

RandomForestClassifier
Best Score: 1.0
Best Parameters: {'rf__n_estimators': 200, 'rf__max_features': 'log2', 'rf__max_depth': 7, 'rf__criterion': 'gini'}
ROC A